In [21]:
import numpy as np
import pandas as pd
import gc
gc.collect()



path = './data/'
player_ext = 'Players_2019.csv'
event_ext = 'Events_2019.csv'
teams_ext = 'Teams.csv'

events = pd.read_csv(path+event_ext)
players = pd.read_csv(path+player_ext)

In [22]:
events['GameID'] = events['Season'].astype(str) + events['DayNum'].astype(str) + events['WTeamID'].astype(str) + events['LTeamID'].astype(str)
events['OT'] = (events['ElapsedSeconds'] - 2400)/300
events['OT'] = np.where(events['OT'] < 0, 0, np.ceil(events['OT'])).astype(int)
events['TimeRemain'] = 2400 + (events['OT'] * 300) - events['ElapsedSeconds']




In [23]:
one_point = ['made1_free']
two_point = ['made2_dunk','made2_jump','made2_lay','made2_tip']
three_point = ['made3_jump']
rebound = ['reb_def','reb_dead','reb_off']
assist = ['assist']
block = ['block']
turnover = ['turnover']
steal = ['steal']

fantasy_events = one_point + two_point + three_point + rebound + assist + block + turnover + steal

fp = events.loc[events['EventType'].isin(fantasy_events)]

fp.loc[fp['EventType'].isin(two_point), 'EventType'] = '2PT'
fp.loc[fp['EventType'].isin(rebound), 'EventType'] = 'Reb'

pgb = fp.groupby(['EventPlayerID','EventType'])['EventID'].count().unstack(level=-1).reset_index()
# pgb = pgb.rename(columns={'EventPlayerID':'PlayerID'})
pgb = pgb.fillna(0)


tgb = fp.groupby(['EventTeamID','EventType'])['EventID'].count().unstack(level=-1).reset_index()

# tgb = tgb.rename(columns={'FGA':'TFGA','FTA':'TFTA','turnover':'TTO','EventTeamID':'TeamID'})
tgb.head()

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


EventType,EventTeamID,2PT,Reb,assist,block,made1_free,made3_jump,steal,turnover
0,1101,525,994,424,74,400,210,232,338
1,1102,542,1104,412,56,330,225,149,409
2,1103,467,1194,370,97,360,275,174,369
3,1104,594,1356,401,154,482,233,152,449
4,1105,535,1077,379,48,277,180,221,477


In [24]:
list(events)

['EventID',
 'Season',
 'DayNum',
 'WTeamID',
 'LTeamID',
 'WPoints',
 'LPoints',
 'ElapsedSeconds',
 'EventTeamID',
 'EventPlayerID',
 'EventType',
 'GameID',
 'OT',
 'TimeRemain']

In [25]:
# need total games played
tgp = events.groupby(['EventTeamID','GameID'])['OT'].max().reset_index()

ot = tgp.groupby(['EventTeamID'])['OT'].sum().reset_index()

tgp = tgp.groupby(['EventTeamID'])['GameID'].count().reset_index()

tgp = tgp.rename(columns={'GameID':'GameCount'})
ot = ot.rename(columns={'OT':'OTCount'})

ot.OTCount.median()

2.0

In [26]:
# player games played
pgp = events.groupby(['EventPlayerID','GameID'])['OT'].max().reset_index()
pgp = pgp.groupby(['EventPlayerID'])['GameID'].count().reset_index()
pgp = pgp.rename(columns={'GameID':'GameCount'})
pgp.head()

,EventPlayerID,GameCount
0,653508,21
1,653509,13
2,653510,26
3,653511,29
4,653512,29


In [27]:
tgb = pd.merge(tgb, tgp, how='left',on=['EventTeamID','EventTeamID'])
tgb.head()

,EventTeamID,2PT,Reb,assist,block,made1_free,made3_jump,steal,turnover,GameCount
0,1101,525,994,424,74,400,210,232,338,29
1,1102,542,1104,412,56,330,225,149,409,31
2,1103,467,1194,370,97,360,275,174,369,31
3,1104,594,1356,401,154,482,233,152,449,33
4,1105,535,1077,379,48,277,180,221,477,31


In [28]:
tgb['Team_FPTS'] = 2*tgb['2PT'] + 1.25 * tgb['Reb'] + 1.5 * tgb['assist'] + 2 * tgb['block'] + tgb['made1_free'] + 3.5*tgb['made3_jump'] + 2*tgb['steal'] - 0.5*tgb['turnover']
# add small OT factor
tgb['TPPG'] = tgb['Team_FPTS']/(tgb['GameCount']+0.25)

tgb['TPPG'] = tgb['TPPG'].round(2)
tgb = tgb[['EventTeamID','TPPG']]
tgb.head()


,EventTeamID,TPPG
0,1101,154.07
1,1102,140.96
2,1103,149.17
3,1104,155.47
4,1105,134.12


In [29]:
pgb = pd.merge(pgb, pgp, how='left',on=['EventPlayerID','EventPlayerID'])


In [30]:
pgb['Player_FPTS'] = 2*pgb['2PT'] + 1.25 * pgb['Reb'] + 1.5 * pgb['assist'] + 2 * pgb['block'] + pgb['made1_free'] + 3.5*pgb['made3_jump'] + 2*pgb['steal'] - 0.5*pgb['turnover']
# add small OT factor
pgb['FPPG'] = pgb['Player_FPTS']/(pgb['GameCount']+0.25)

pgb['FPPG'] = pgb['FPPG'].round(2)
pgb = pgb.sort_values(by=['FPPG'],ascending=False)
pgb.head()



,EventPlayerID,2PT,Reb,assist,block,made1_free,made3_jump,steal,turnover,GameCount,Player_FPTS,FPPG
2620,656216,188.0,164.0,290.0,24.0,199.0,49.0,47.0,155.0,29,1451.00,49.61
892,654429,322.0,361.0,64.0,71.0,126.0,7.0,48.0,105.0,32,1527.25,47.36
3458,657082,203.0,363.0,55.0,30.0,186.0,64.0,24.0,68.0,31,1426.25,45.64
554,654080,142.0,143.0,85.0,9.0,227.0,124.0,43.0,72.0,29,1319.25,45.10
1834,655408,166.0,256.0,163.0,30.0,126.0,45.0,63.0,90.0,30,1321.00,43.67


In [31]:
players = players.drop(columns=['Season'])

#rename
pgb = pgb.rename(columns={'EventPlayerID':'PlayerID'})
tgb = tgb.rename(columns={'EventTeamID':'TeamID'})

pgb = pd.merge(left=pgb, right=players, how='left', on=['PlayerID','PlayerID'])
pgb = pd.merge(left=pgb, right=tgb, how='left', on=['TeamID','TeamID'])
pgb.head()



,PlayerID,2PT,Reb,assist,block,made1_free,made3_jump,steal,turnover,GameCount,Player_FPTS,FPPG,TeamID,PlayerName,TPPG
0,656216,188.0,164.0,290.0,24.0,199.0,49.0,47.0,155.0,29,1451.00,49.61,1293,JA_MORANT,177.03
1,654429,322.0,361.0,64.0,71.0,126.0,7.0,48.0,105.0,32,1527.25,47.36,1169,LAMINE_DIANE,164.98
2,657082,203.0,363.0,55.0,30.0,186.0,64.0,24.0,68.0,31,1426.25,45.64,1355,MIKE_DAUM,171.93
3,654080,142.0,143.0,85.0,9.0,227.0,124.0,43.0,72.0,29,1319.25,45.10,1144,CHRIS_CLEMONS,152.09
4,655408,166.0,256.0,163.0,30.0,126.0,45.0,63.0,90.0,30,1321.00,43.67,1236,JOHN_KONCHAR,169.50


In [32]:
fp = pgb[['TeamID','PlayerID','PlayerName','FPPG','TPPG']]

teams_ext = 'Teams.csv'
teams = pd.read_csv(path+teams_ext)
teams = teams[['TeamID','TeamName']]

fp = pd.merge(fp, teams, how='left',on=['TeamID','TeamID'])

fp = fp[['TeamName','PlayerName','FPPG']]
fp.head()

,TeamName,PlayerName,FPPG
0,Murray St,JA_MORANT,49.61
1,CS Northridge,LAMINE_DIANE,47.36
2,S Dakota St,MIKE_DAUM,45.64
3,Campbell,CHRIS_CLEMONS,45.10
4,IPFW,JOHN_KONCHAR,43.67


In [33]:
needed_teams = ['Clemson','Virginia Tech','Duke','Kansas','Fresno St','Oregon','Georgia Tech','NC State',
                'Michigan St','Kentucky','Nevada','Utah',"St Mary's CA",'Wisconsin']
fp = fp.loc[fp.TeamName.isin(needed_teams)]
print(len(fp.TeamName.unique()))


14


In [34]:
rosters = pd.read_csv('2019_rosters.csv')
rosters = rosters.drop(columns=['Unnamed: 0','number'])

rosters.head()


,name,position,height,weight,class,hometown,Team
0,Clyde Trapp,G,"6' 4""",194 lbs,JR,"Eastover, SC",Clemson
1,Jonathan Baehre,F,"6' 10""",209 lbs,JR,"Wiesbaden, Germany",Clemson
2,Al-Amir Dawes,G,"6' 2""",180 lbs,FR,"Newark, NJ",Clemson
3,Chase Hunter,G,"6' 3""",200 lbs,FR,"Atlanta, GA",Clemson
4,Nick Honor,G,"5' 10""",205 lbs,SO,"Orlando, FL",Clemson


In [42]:
print(rosters.Team.unique())
print(fp.TeamName.unique())

team_trans = {
    'Michigan State':'Michigan St',
    "Saint Mary's":"St Mary's CA",
    "Fresno State":'Fresno St'
}


['Clemson' 'Virginia Tech' 'Duke' 'Kansas' 'Fresno State' 'Oregon'
 'Georgia Tech' 'NC State' 'Michigan State' 'Kentucky' 'Nevada' 'Utah'
 "Saint Mary's" 'Wisconsin']
['Duke' 'Wisconsin' 'Oregon' 'Kansas' 'Michigan St' 'Clemson' 'Nevada'
 'Fresno St' 'Virginia Tech' "St Mary's CA" 'Kentucky' 'Georgia Tech'
 'Utah' 'NC State']


In [35]:
fp.loc[:,'PlayerName'] = fp['PlayerName'].str.replace("_"," ")
fp.loc[:,'PlayerName'] = fp['PlayerName'].str.title()

fp.head()


,TeamName,PlayerName,FPPG
7,Duke,Zion Williamson,43.22
12,Duke,Rj Barrett,40.29
14,Wisconsin,Ethan Happ,40.20
16,Oregon,Bol Bol,39.95
25,Kansas,Dedric Lawson,38.25


In [36]:
roster_to_fp = {
    "Alex O'Connell":'Alex Oconnell',
    "C.J. Bryce":'Cj Bryce',
    "D'Mitrik Trice":"Dmitrik Trice",
    "DJ Funderburk":'Derek Funderburk',
    "David McCormack":'David Mccormack',
    "EJ Montgomery":'Ej Montgomery',
    "James Banks III":'James Banks Iii',
    "John Newman III":'John Newman Iii',
    "Marcus Bingham Jr.":'Marcus Bingham Jr',
    "P.J. Horne":'Pj Horne',
    "Walt McGrory":'Walt Mcgrory'
}

fp_to_roster = dict(map(reversed, roster_to_fp.items()))
fp = fp.replace({"PlayerName": fp_to_roster})


In [37]:
# ### check for similar names to match 
# roster_names = list(rosters.name.values)
# use_names = list(use.PlayerName.values)

# print("Use minus roster")
# udiff = list(set(use_names)-set(roster_names))
# udiff.sort()
# print(udiff)

# print("Roster minus use")
# rdiff = list(set(roster_names)-set(use_names))
# rdiff.sort()
# print(rdiff)

# import difflib

# for rword in rdiff:
#     print(rword, difflib.get_close_matches(rword,udiff))

In [38]:
roster_names = list(rosters.name.values)
print(roster_names)
fp['Graduated'] = np.where(fp['PlayerName'].isin(roster_names),"Still On Roster","Lost")


['Clyde Trapp', 'Jonathan Baehre', 'Al-Amir Dawes', 'Chase Hunter', 'Nick Honor', 'Hunter Tyson', 'Curran Scott', 'Parker Fox', 'Alex Hemenway', 'Tevin Mack', 'John Newman III', "O'Neil McBride", 'Khavon Moore', 'Aamir Simms', 'Paul Grinde', 'Trey Jemison', 'Hunter Cattoor', 'Isaiah Wilkins', 'Landers Nolley II', 'Wabissa Bede', 'Jonathan Kabongo', 'Brendan Palmer', 'Grant Yates', 'P.J. Horne', 'Jalen Cone', 'John Ojiako', 'Keve Aluma', 'Tyrece Radford', 'Branden Johnson', 'Ryan Payne', 'Wendell Moore Jr.', 'Vernon Carey Jr.', 'Cassius Stanley', 'Tre Jones', 'Javin DeLaurier', 'Joey Baker', 'Jordan Goldwire', "Alex O'Connell", 'Matthew Hurt', 'Michael Savarino', 'Jack White', 'Keenan Worthington', 'Justin Robinson', 'Mike Buckmire', 'Marcus Garrett', 'Devon Dotson', 'Christian Braun', 'Dajuan Harris', 'Isaiah Moss', 'Elijah Elliott', 'Jalen Wilson', 'Chris Teahan', 'Tristan Enaruna', 'Michael Jankovich', 'Silvio De Sousa', 'Ochai Agbaji', 'David McCormack', 'Udoka Azubuike', 'Mitch Lig

In [40]:
print(fp.loc[fp['PlayerName']=='Justin Robinson'])

           TeamName       PlayerName   FPPG        Graduated
308   Virginia Tech  Justin Robinson  28.02  Still On Roster
4164           Duke  Justin Robinson   2.66  Still On Roster


In [19]:
fgb = fp.groupby(['TeamName','Graduated'])['FPPG'].sum().unstack(-1).reset_index()
fgb.sort_values(by="Still On Roster", ascending=False)


Graduated,TeamName,Lost,Still On Roster
6,Michigan St,78.97,129.48
10,St Mary's CA,30.95,122.42
7,NC State,68.56,119.82
3,Georgia Tech,54.26,114.71
4,Kansas,99.17,103.65
13,Wisconsin,70.07,88.84
2,Fresno St,118.04,82.22
0,Clemson,85.07,73.27
12,Virginia Tech,109.42,61.30
1,Duke,152.17,58.14


In [20]:
returning = fp.loc[fp.Graduated=="Still On Roster"]
returning = returning.drop(columns=['Graduated'])
fgb = fgb.rename(columns={'Lost':'Available FPPG'})
fgb = fgb.drop(columns=['Still On Roster'])
returning = pd.merge(returning, fgb, how='left',on=['TeamName','TeamName'])
returning = returning.sort_values(by=['FPPG'],ascending=False)
print(returning.head(50))

         TeamName         PlayerName   FPPG  Available FPPG
0     Michigan St    Cassius Winston  35.75           78.97
1    St Mary's CA        Jordan Ford  31.02           30.95
2         Clemson      Elijah Thomas  29.14           85.07
3    St Mary's CA        Malik Fitts  28.81           30.95
4       Fresno St        Nate Grimes  28.32          118.04
5   Virginia Tech    Justin Robinson  28.02          109.42
6          Oregon   Payton Pritchard  27.67          151.67
7    Georgia Tech    James Banks III  27.11           54.26
8            Duke          Tre Jones  25.73          152.17
9    Georgia Tech      Jose Alvarado  25.69           54.26
10    Michigan St     Xavier Tillman  25.28           78.97
11         Kansas     Udoka Azubuike  25.05           99.17
12    Michigan St    Joshua Langford  24.14           78.97
13         Kansas       Devon Dotson  24.07           99.17
14       NC State    Markell Johnson  23.26           68.56
15           Utah        Timmy Allen  23